### video

In [1]:
!ls -Sr ~/videos
#VIDEO = 'sample_peds.mp4'
VIDEO = 'pred_feed.vob'
# VIDEO = 'desert.mp4'
# VIDEO = 'rabbit.mpg'
# VIDEO = 'sample_faces5.mpg'
# VIDEO = 'sao_paulo_1.mpg'
# VIDEO = 'desert_1.mp4'
# VIDEO = 'sample.mpg'
# VIDEO = 'sample_motion6.mpg'

# VIDEO = 'sample_contour3.mpg' 

PATH_TO_VIDEO = '/Users/will/videos/' + VIDEO

sample_contour3.mpg rabbit.mpg          desert.mp4          desert_1.mp4
sample.mpg          sample_faces5.mpg   pred_feed.vob
sample_motion6.mpg  sample_peds.mp4     sao_paulo_1.mpg


### model

In [8]:
# model
MODEL = '/Users/will/git/python-streams-events/src/ssd_mobilenet_v1_coco_2017_11_17/frozen_inference_graph.pb'
# MODEL = '/Users/will/git/models/research/gan/mnist/data/classify_mnist_graph_def.pb'
# MODEL = '/Users/will/git/models/research/object_detection/test_ckpt/ssd_inception_v2.pb'
# MODEL = '/Users/will/git/models/samples/languages/java/training/model/graph.pb'
# MODEL = '/Users/will/git/python-streams-events/src/ssd_mobilenet_v1_coco_2017_11_17/frozen_inference_graph.pb'

### params

In [9]:
# objects
NUM_CLASSES = 100
PATH_TO_LABELS = '/Users/will/git/models/research/object_detection/data/'+'mscoco_label_map.pbtxt'

# performance
LINE_THICKNESS = 1
PROBABILITY_THRESHHOLD = 0.10
RECORDING_FPS = 15

### imports

In [10]:
import tensorflow as tf
import numpy as np
import cv2
print('opencv ver: ', cv2.__version__)

import sys
import os
from collections import Counter

sys.path.append("/Users/will/git/models/research")
sys.path.append("/Users/will/git/models/research/object_detection")
from utils import label_map_util
from utils import visualization_utils as vis_util

opencv ver:  3.4.1


### object detect

In [11]:
def process_video(video, model, labels, num_classes, line_thickness, probability_cutoff, fps):
    
    object_counts = []
    cap = cv2.VideoCapture(video)
    # codec = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    # vidWriter = cv2.VideoWriter('output_path', codec, fps, (int(cap.get(3)), int(cap.get(4))))

    # Load a (frozen) Tensorflow model into memory.
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(model, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    
    # Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine
    label_map = label_map_util.load_labelmap(labels)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES,use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as tf_sess:
            ret = True
            while (ret):
                ret, image_np = cap.read()
                if (ret == False):
                    break

                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
                detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
                num_detections = detection_graph.get_tensor_by_name('num_detections:0')
                image_np_expanded = np.expand_dims(image_np, axis=0)

                # Actual detection.
                fd = {image_tensor: image_np_expanded}
                detection_info = [detection_boxes, detection_scores, detection_classes, num_detections]
                (boxes, scores, classes, num) = tf_sess.run(detection_info, fd)
                # Visualization of the results of a detection
                vis_util.visualize_boxes_and_labels_on_image_array(
                    image_np,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness = LINE_THICKNESS
                )

                # get list of objects and their counts
                len_sqzd_scrs =  len(list(filter(lambda x: x > probability_cutoff, np.squeeze(scores))))
                obj_cnt = [(category_index.get(tpl[0])['name'], tpl[1]) for tpl in Counter(np.squeeze(classes)[:len_sqzd_scrs]).items()]

                object_counts.append(obj_cnt)

                # vidWriter.write(image_np)
                cv2.imshow('live_detection', image_np)
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    exit(0)                

### EXPERIMENT

In [12]:
def process_video(video, MODEL, labels, num_classes, line_thickness, probability_cutoff, fps):
    
    object_counts = []
    cap = cv2.VideoCapture(video)
    # codec = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    # vidWriter = cv2.VideoWriter('output_path', codec, fps, (int(cap.get(3)), int(cap.get(4))))

    # Load a (frozen) Tensorflow model into memory.
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(MODEL, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    
    # Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine
    label_map = label_map_util.load_labelmap(labels)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES,use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    
    with tf.Session(graph=detection_graph) as tf_sess:
        ret = True
        while (ret):
            ret, image_np = cap.read()
            if (ret == False):
                break

            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            image_np_expanded = np.expand_dims(image_np, axis=0)

            # Actual detection.
            fd = {image_tensor: image_np_expanded}
            detection_info = [detection_boxes, detection_scores, detection_classes, num_detections]
            (boxes, scores, classes, num) = tf_sess.run(detection_info, fd)
            # Visualization of the results of a detection
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness = LINE_THICKNESS
            )

            # get list of objects and their counts
            len_sqzd_scrs =  len(list(filter(lambda x: x > probability_cutoff, np.squeeze(scores))))
            obj_cnt = [(category_index.get(tpl[0])['name'], tpl[1]) for tpl in Counter(np.squeeze(classes)[:len_sqzd_scrs]).items()]

            object_counts.append(obj_cnt)

            # vidWriter.write(image_np)
            cv2.imshow('live_detection', image_np)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                exit(0)                

In [ ]:
process_video(
    PATH_TO_VIDEO, 
    MODEL, 
    PATH_TO_LABELS, 
    NUM_CLASSES, 
    LINE_THICKNESS, 
    PROBABILITY_THRESHHOLD, 
    RECORDING_FPS)